# 环境配置

In [1]:
!pip install albumentations

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 72 kB 6.6 MB/s eta 0:00:011
     |████████████████████████████████| 12.4 MB 477 kB/s eta 0:00:01�███| 12.4 MB 477 kB/s eta 0:00:01
     |████████████████████████████████| 948 kB 104.0 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 451 kB/s eta 0:00:011
     |████████████████████████████████| 3.3 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 90.0 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 16.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 67.0 MB/s eta 0:00:01
     |████████████████████████████████| 51.0 MB 441 kB/s eta 0:00:011��████▊                     | 17.1 MB 956 kB/s eta 0:00:36


In [13]:
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/pytorch_py3

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    attrs-21.2.0               |     pyhd8ed1ab_0          44 KB  conda-forge
    bleach-3.3.0               |     pyh44b312d_0         111 KB  conda-forge
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    defusedxml-0.7.1           |     pyhd8ed1ab_0          23 KB  conda-forge
    importlib-metadata-4.4.0   |   py36h5fab9bb_0          31 KB  conda-forge
    ipywidgets-7.6.3           |     pyhd3deb0d_0         101 KB  conda-f

In [6]:
!pip install transformers

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 2.2 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 722 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 15.3 MB/s eta 0:00:01


# 提取frame

In [1]:
import cv2
import sys
import numpy as np
import torch
import torch.nn as nn
def frame_iterator_list(filename, every_ms=1000, max_num_frames=300):
    video_capture = cv2.VideoCapture()
    if not video_capture.open(filename):
        print(sys.stderr, 'Error: Cannot open video file ' + filename)
        return
    last_ts = -99999  # The timestamp of last retrieved frame.
    num_retrieved = 0

    frame_all = []
    while num_retrieved < max_num_frames:
        # Skip frames
        while video_capture.get(cv2.CAP_PROP_POS_MSEC) < every_ms + last_ts:
            if not video_capture.read()[0]:
                return np.array(frame_all)

        last_ts = video_capture.get(cv2.CAP_PROP_POS_MSEC)
        has_frames, frame = video_capture.read()
        if not has_frames:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_all.append(frame[:, :, ::-1])
        num_retrieved += 1

    return np.array(frame_all)

In [16]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms as T
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform
transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
A_transform = A.Compose([
            # Resize(CFG.image_size, CFG.image_size),
            # A.RandomResizedCrop(256, 256),
            # A.Transpose(p=0.5),
            # A.HorizontalFlip(p=0.5),
            # A.VerticalFlip(p=0.5),
            # A.ShiftScaleRotate(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],  # mean on ImageNet
                std=[0.229, 0.224, 0.225],  # std on ImageNet
            ),
            ToTensorV2(),
        ])

In [3]:
import os
import cv2
video_path = '../dataset/videos/video_5k/train_5k/'
file_names = os.listdir(video_path)
save_path = '../dataset/frames/train_5k/'

In [4]:
# every_ms 指的是每 every_ms 提取 1帧，every_ms=1000时，每秒提取 1帧
frame_list = frame_iterator_list(video_path+file_names[0],every_ms=1000)

In [5]:
video_id = file_names[0][:-4]
save_name = save_path+video_id+'.npy'

In [6]:
import os
for frame in frame_list:
    video_id = file_names[0][:-4]
    save_name = save_path+video_id+'.npy'
    if(os.path.exists(save_name)):
        print(f'{file_names[0]} is exit')
        break
    frame_list = frame_iterator_list(video_path+file_names[0],every_ms=1000)
    #np.save(save_name,frame_list)
    break

NameError: name 'frame_list' is not defined

In [27]:
%%time
video_processed = []
for frame in frame_list:
    augmented = A_transform(image=frame)
    video_processed.append(augmented['image'].unsqueeze(0))

CPU times: user 92 ms, sys: 12 ms, total: 104 ms
Wall time: 103 ms


In [28]:
video = torch.cat(video_processed,dim = 0)

In [29]:
video.shape

torch.Size([56, 3, 256, 256])

In [8]:
!pip install timm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 346 kB 10.0 MB/s eta 0:00:01


In [9]:
import timm
timm.list_models()

['adv_inception_v3',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'eca_nfnet_l0',
 'eca_nfnet_l1',
 'eca_vovnet39b',
 'ecaresnet26t'

In [12]:
# 用effecientnet试试
import timm
class CustomResNext(nn.Module):
    def __init__(self):
        super(CustomResNext, self).__init__()
        self.model = timm.create_model('resnext50_32x4d',pretrained=True)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Identity()
    def forward(self,x):
        x = self.model(x)
        return x
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.model = models.resnet50(pretrained=True)
        
    def forward(self, input):
        output = self.model.conv1(input)
        output = self.model.bn1(output)
        output = self.model.relu(output)
        output = self.model.maxpool(output)
        output = self.model.layer1(output)
        output = self.model.layer2(output)
        output = self.model.layer3(output)
        output = self.model.layer4(output)
        output = self.model.avgpool(output)
        return output

In [13]:
model = CustomResNext()
model.eval()

CustomResNext(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

In [19]:
for frame in frame_list:
    augmented = A_transform(image = frame)
    frame = augmented['image']
    pred = model(frame.unsqueeze(0))
    pred = pred.view(-1)
    break

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [18]:
pred.shape

torch.Size([2048])

In [11]:
np.savez_compressed("./a", frame_list)

In [30]:
%%time
a = np.load('./a.npz')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 854 µs


In [29]:
%%time
with open('a.npz','rb') as f:
    a = np.load(f)
    b = a['arr_0']

CPU times: user 748 ms, sys: 68 ms, total: 816 ms
Wall time: 816 ms


In [26]:
b.shape

(56, 720, 1280, 3)

# 端到端训练

## dataloader

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from transformers import BertTokenizer
import os
import linecache
import re
from torch.nn.utils.rnn import pad_sequence
import cv2
import numpy as np
class MultimodaRawDataset(Dataset):

    def __init__(self,dataset_config,job='training'):
        
        self.text_max_len = dataset_config['text_max_len']
        self.device = dataset_config['device']
        self.data_num_per_sample = 4
        if(job=='training'):
            self.meta_path = dataset_config['train_raw_data_path']
        elif(job=='valdation'):
            self.meta_path = dataset_config['val_raw_data_path']
        else:
            self.meta_path = dataset_config['test_data_path']
        self.tokenizer = BertTokenizer.from_pretrained(dataset_config['bert_path'])
        self.label2id = {}
        with open(dataset_config['label_id_path'],'r') as f:
            for line in f:
                line = line.strip('\r\n')
                line = line.split('\t')
                self.label2id[line[0]] = int(line[1])
        self.A_transform = A.Compose([
                # Resize(CFG.image_size, CFG.image_size),
                A.RandomResizedCrop(256, 256),
                A.Transpose(p=0.5),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.Normalize(
                    mean=[0.485, 0.456, 0.406],  # mean on ImageNet
                    std=[0.229, 0.224, 0.225],  # std on ImageNet
                ),
                ToTensorV2(),
            ])
    def __getitem__(self, index):
        # 1. 从train.txt读取对应 idx 的path
        data_list = [] # 存储对于index的各个模态数据的路径和样本标签
        for line_i in range(self.data_num_per_sample*index+1,self.data_num_per_sample*(index+1)):
            line = linecache.getline(self.meta_path,line_i)
            line = line.strip('\r\n')
            data_list.append(line)
        #print(data_list)
        video,text_ids,text_attention_mask,label_ids = self.preprocess(data_list)
        return video,text_ids,text_attention_mask,label_ids
    def __len__(self):
        # TODO 不能固定长度
        with open(self.meta_path,'r') as f:
            lines = f.readlines()
        return len(lines)//self.data_num_per_sample
    def preprocess(self,data_list):
        
        video_path,text_path,label = data_list
        
        #--------------- video ----------------#
        
        frame_list = self.frame_iterator_list(video_path)
        
        video = []
        for frame in frame_list:
            augmented = self.A_transform(image=frame)
            video.append(augmented['image'].unsqueeze(0))
        video = torch.cat(video,dim=0)# shape(len,channel,H,W)
        #--------------- text ----------------#
        text = ''
        with open(text_path,'r') as f:
            for line in f:
                dic = eval(line)
           
        for key in dic:
            dic[key] = ''.join(re.findall('[\u4e00-\u9fa5]',dic[key]))
            text += dic[key]
        
        # text = ''.join(re.findall('[\u4e00-\u9fa5]',dic['video_asr']))
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            truncation=True,
            max_length=self.text_max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        text_ids = inputs['input_ids']
        text_attention_mask = inputs['attention_mask']
        text_ids = torch.tensor(np.array(text_ids).astype('int64'))
        text_attention_mask = torch.tensor(np.array(text_attention_mask).astype('int64'))
        #--------------- label ----------------#
        label_ids = []
        label = label.split(',')
        np.random.shuffle(label)
        for i in label:
            label_ids.append(self.label2id[i])
        # label_ids = torch.tensor(np.array(label_ids).astype('int64'))
        dense_label_ids = torch.zeros(82)# ,dtype=torch.int64)
        dense_label_ids[label_ids] = 1
        # return video,audio,label_ids
        return video,text_ids,text_attention_mask,dense_label_ids
    
    def collate_fn(self,batch):
        # 自定义dataloader 对一个batch的处理方式
        # 需要完成的任务有：
        # 1. 对video和audio的序列进行padding
        # 2. 对text，label_ids同样padding
        video_stacks = []
        text_stacks = []
        label_stacks = []
        text_attention_stacks = []
        for i in batch:
            video_stacks.append(i[0])
            text_stacks.append(i[1])
            text_attention_stacks.append(i[2])
            label_stacks.append(i[3])
        
        video_stacks = pad_sequence(video_stacks,batch_first=True,padding_value=0)
        text_stacks = pad_sequence(text_stacks,batch_first=True,padding_value=0) # 实际上没有pad
        # 实际上并没有padding，因为label变成multi-hot向量，长度都是82
        label_stacks = pad_sequence(label_stacks,batch_first=True,padding_value=0) 
        text_attention_stacks = pad_sequence(text_attention_stacks,batch_first=True,padding_value=0) # 实际上也没有pad
        return video_stacks,text_stacks,text_attention_stacks,label_stacks
    
    def frame_iterator_list(self,filename, every_ms=1000, max_num_frames=300):
        video_capture = cv2.VideoCapture()
        if not video_capture.open(filename):
            print(sys.stderr, 'Error: Cannot open video file ' + filename)
            return
        last_ts = -99999  # The timestamp of last retrieved frame.
        num_retrieved = 0

        frame_all = []
        while num_retrieved < max_num_frames:
            # Skip frames
            while video_capture.get(cv2.CAP_PROP_POS_MSEC) < every_ms + last_ts:
                if not video_capture.read()[0]:
                    return frame_all

            last_ts = video_capture.get(cv2.CAP_PROP_POS_MSEC)
            has_frames, frame = video_capture.read()
            if not has_frames:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_all.append(frame[:, :, ::-1])
            num_retrieved += 1

        return frame_all

In [8]:
import yaml
from torch.utils.data import DataLoader
config_path = './config/config.yaml'
config = yaml.load(open(config_path))
train_dataset = MultimodaRawDataset(config['DatasetConfig'],job='training')
dataloader = DataLoader(train_dataset,batch_size=4,num_workers=4,collate_fn=train_dataset.collate_fn)

/opt/conda/envs/pytorch_py3/lib/python3.6/site-packages/ipykernel/__main__.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [ ]:
it = iter(dataloader)
batch = next(it)

In [10]:
video,text_ids,text_attention_mask,dense_label_ids = batch

In [ ]:
video.shape

In [9]:
from src.video_head.nextvlad import NeXtVLAD
import torchvision.models as models
import torch.nn as nn
class RawNeXtVLAD(nn.Module):
    def __init__(self,feature_size, max_frames, nextvlad_cluster_size, expansion, groups):
        super(RawNeXtVLAD,self).__init__()
        self.nextvlad = NeXtVLAD(feature_size, max_frames, nextvlad_cluster_size, expansion, groups)
        self.resnet50 = models.resnet50(pretrained=True)
        self.resnet50.fc = nn.Linear(2048,1024)
        
    def forward(self,input,mask=None):
        # 输入图像shape (batch,len,channel,H,W)
        B,S,C,H,W = input.shape
        input = input.contiguous().view(B*S,C,H,W)
        output = self.resnet50(input)
        output = output.contiguous().view(B,S,-1)
        if(mask!=None):
            output = self.nextvlad(output,mask)
        else:
            output = self.nextvlad(output)
        return output

In [ ]:
model = RawNeXtVLAD(1024,300,128,2,16)
pred = model(video)

In [11]:
pred.shape

torch.Size([4, 16384])

In [13]:
video.shape

torch.Size([4, 57, 3, 256, 256])

In [15]:
def  _modal_drop( x, rate=0.0, noise_shape=None):
    """模态dropout"""
    random_scale = torch.rand(noise_shape)
    keep_mask = (random_scale >= rate).type(x.dtype).to(x.device) # >= rate的才保留
    ret = x * keep_mask
    probs = keep_mask.type(torch.float32) # cast将张量进行类型转换
    return ret, probs

In [16]:
inputs_dict, prob_dict = _modal_drop(video, 0.2, [4,1,1,1,1])

In [22]:
mask = (video != 0).type(torch.float32).sum(dim=-1).sum(dim=-1).sum(dim=-1).type(torch.bool)

In [23]:
mask.shape

torch.Size([4, 57])

In [5]:
# 生成raw_train_good.txt和raw_val_good.txt
import os
import linecache
train_good_path = '../dataset/tagging/GroundTruth/datafile/val_good.txt'
raw_train_good_path = '../dataset/tagging/GroundTruth/datafile/raw_val_good.txt'
dataset_path = '../dataset/videos/video_5k/train_5k/'
with open(raw_train_good_path,'w') as f:
    for index in range(4500):
        video_i = 6*index+1
        text_i = 6*index+4
        label_i = 6*index+5
        video_line = linecache.getline(train_good_path,video_i)
        video_line = video_line.strip('\r\n')
        video_id = os.path.basename(video_line)[:-4]
        video_raw_path = dataset_path+video_id+'.mp4'+'\r\n'
        text_line = linecache.getline(train_good_path,text_i)
        label_line = linecache.getline(train_good_path,label_i)
        f.write(video_raw_path)
        f.write(text_line)
        f.write(label_line)
        f.write('\r\n')

# 重新提取特征

In [ ]:
import cv2
import sys
import numpy as np
import torch
import torch.nn as nn
def frame_iterator_list(filename, every_ms=1000, max_num_frames=300):
    video_capture = cv2.VideoCapture()
    if not video_capture.open(filename):
        print(sys.stderr, 'Error: Cannot open video file ' + filename)
        return
    last_ts = -99999  # The timestamp of last retrieved frame.
    num_retrieved = 0

    frame_all = []
    while num_retrieved < max_num_frames:
        # Skip frames
        while video_capture.get(cv2.CAP_PROP_POS_MSEC) < every_ms + last_ts:
            if not video_capture.read()[0]:
                return np.array(frame_all)

        last_ts = video_capture.get(cv2.CAP_PROP_POS_MSEC)
        has_frames, frame = video_capture.read()
        if not has_frames:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_all.append(frame[:, :, ::-1])
        num_retrieved += 1

    return np.array(frame_all)